In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：11


['http://www.fenderliner.com/products-show.asp?seq=4650',
 'http://www.fenderliner.com/products-show.asp?seq=4599',
 'http://www.fenderliner.com/products-show.asp?seq=4882',
 'http://www.fenderliner.com/products-show.asp?seq=4884',
 'http://www.fenderliner.com/products-show.asp?seq=6063',
 'http://www.fenderliner.com/products-show.asp?seq=6099',
 'http://www.fenderliner.com/products-show.asp?seq=6027',
 'http://www.fenderliner.com/products-show.asp?seq=6028',
 'http://www.fenderliner.com/products-show.asp?seq=6172',
 'http://www.fenderliner.com/products-show.asp?seq=6173',
 'http://www.fenderliner.com/products-show.asp?seq=6042']

In [3]:
crawler_status = 'error'

crawler_status

'error'

In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get(input_.loc[a, 'Url'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'\r\n<!DOCTYPE html>\r\n<html xmlns="http://www.w3.org/1999/xhtml" vocab="http://schema.org/">\r\n<head>\r\n\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n<title>Fender Liner Manufacturer in Taiwan│GBC</title>\r\n<meta name="viewport" content="user-scalable=1,width=1200,initial-scale=1">\r\n<meta http-equiv="content-language" content="en">\r\n<meta name="description" property="og:description"  content="Fender Liner Manufacturer in Taiwan│GBC" />\r\n<meta name="keywords" content="Fender Liner Manufacturer in Taiwan│GBC" />\r\n<meta property="og:title" content="Fender Liner Manufacturer in Taiwan│GBC"/>\r\n<meta property="og:url" content=""/>\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:image" content=""/>\r\n<meta property="og:site_name" content=""/>\r\n<meta name="author" content="" />\r\n<meta name="copyright" content="" />\r\n<link rel="Shortcut icon" href="images/favicon.ico" type="image/x-icon" />\r\n<link rel="Bookmark" href="

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

In [7]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html vocab="http://schema.org/" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Fender Liner Manufacturer in Taiwan│GBC
  </title>
  <meta content="user-scalable=1,width=1200,initial-scale=1" name="viewport"/>
  <meta content="en" http-equiv="content-language"/>
  <meta content="Fender Liner Manufacturer in Taiwan│GBC" name="description" property="og:description"/>
  <meta content="Fender Liner Manufacturer in Taiwan│GBC" name="keywords"/>
  <meta content="Fender Liner Manufacturer in Taiwan│GBC" property="og:title"/>
  <meta content="" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content="" property="og:image"/>
  <meta content="" property="og:site_name"/>
  <meta content="" name="author"/>
  <meta content="" name="copyright"/>
  <link href="images/favicon.ico" rel="Shortcut icon" type="image/x-icon"/>
  <link href="images/favicon.ico" rel="Bookmark" type="i

In [8]:
title = html.xpath('//h3[@class="title01"]/text()')[0].strip()

title

'BMW\xa0X3\xa011-17\xa0(51 71 7 213 641)'

In [9]:
src = 'http://www.fenderliner.com' + html.xpath('//div[@class="b-img"]/img/@src')[0].strip()

src

'http://www.fenderliner.com/img/product/BMX6014A0L-C-1.jpg'

In [10]:
list_ul = html.xpath('//ul[@class="show-info-list"]/li')

list_ul

[<Element li at 0x1156ab400>,
 <Element li at 0x1156ab680>,
 <Element li at 0x1156fad80>,
 <Element li at 0x1156f80c0>,
 <Element li at 0x1156f8140>,
 <Element li at 0x1156f8080>,
 <Element li at 0x1156f8100>,
 <Element li at 0x1156f8280>]

In [11]:
list_h4 = html.xpath('//h4[@class="title03"]')
for h4 in list_h4:
    if h4.xpath('./text()')[0].strip() == 'Description':
        list_li = h4.xpath('./following-sibling::div[@class="edit"]')[0].xpath('./ul[@class="about-list"]/li/text()')

list_li

['PARTS MODEL: BMW X3 F25 /X4 F26',
 'PARTS DESCRIPTION: F.LINER FENDER FT. SECTION LH (CAPA)',
 'YEAR: 2011,2012,2013,2014,2015,2016,2017',
 'OE/AM MATERIAL: PP/PP',
 'OE WEIGHT: 1.08(KG)',
 'AM WEIGHT: 0.86(KG)',
 'OE THICKNESS: 1.85(mm)',
 'AM THICKNESS: 1.95(mm)',
 'OE IZOD IMPACT TEST: 16.2(J)',
 'AM IZOD IMPACT TEST: 17.34(J)',
 'ALSO NAME: FR FENDER LINER / FR FENDER LINER ASSEMBLY/ FR INNER FENDER / FR WATER SPLASH SHIELD/ FR INNER FENDER WELL']

In [12]:
df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                         'Title': title,
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Src': src}])

for ul in list_ul:
    if len(ul.xpath('./@style')) != 0:
        continue
    
    name = ul.xpath('./span/text()')[0].strip()[:-1].strip()
    if name == 'Certifed by':
        df_temp.loc[0, name.upper()] = '' if len(ul.xpath('./img')) == 0 else ';'.join([_.strip() for _ in ul.xpath('./img/@alt')])
    else:
        df_temp.loc[0, name.upper()] = '' if len(ul.xpath('./text()')) == 0 else ul.xpath('./text()')[0].strip()

for li in list_li:
    df_temp.loc[0, li.split(':')[0].strip().lower()] = ' : '.join(_.strip() for _ in li.split(':')[1:])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [13]:
crawler_status = 'ok'

crawler_status

'ok'

In [14]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                             'Url': input_.loc[a, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [15]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,序号,Title,Pic,Url,Src,BRAND,MODEL,YEAR,OE#,PARTSLINK NO.,...,parts description,year,oe/am material,oe weight,am weight,oe thickness,am thickness,oe izod impact test,am izod impact test,also name
0,45,BMW X3 11-17 (51 71 7 213 641),,http://www.fenderliner.com/products-show.asp?s...,http://www.fenderliner.com/img/product/BMX6014...,BMW,X3,11-17,51 71 7 213 641,,...,F.LINER FENDER FT. SECTION LH (CAPA),"2011,2012,2013,2014,2015,2016,2017",PP/PP,1.08(KG),0.86(KG),1.85(mm),1.95(mm),16.2(J),17.34(J),FR FENDER LINER / FR FENDER LINER ASSEMBLY/ FR...


In [16]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
